In [54]:
import pandas as pd
import requests
import time
import re

from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
model_list_urls = [
    'https://auto.ru/cars/used/sale/audi/a6/1102314663-084e0ff8/',
    'https://auto.ru/cars/used/sale/mazda/3/1101836763-72c1fd32/',
    'https://auto.ru/cars/used/sale/ford/mondeo/1101981472-3859561f/',
    'https://auto.ru/cars/used/sale/ford/mondeo/1102310519-7daac9c2/'
]

##  Создаем marks_model_dict
 - словарь в которм ключи это  обозначения марки на сайте auto.ru, а значения это списки обозначений на сайте auto.ru марок каждой модели 

In [2]:
url_for_mmdict = 'https://auto.ru/catalog/cars/all/'

headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1613427368824.604800.BYczsuAhOkO7E_tI9WN3ZQ.vyyvypdKR_sNAGlonPspJxUInUX2GwuD7owqbI5Sw58; X-Vertis-DC=vla; _ym_d=1613250934; from_lifetime=1613250934966; cycada=FXXHRKJxPTj6XyBIJ1I0Or150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobil': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
};

In [4]:
marks_models_dict = {} # создаем словарь с ключами - обозначениеми марки и занчениями - list-ами моделей марки  

for pages_num in range(1,20):
    
    if pages_num==1: params = {'view_type': 'list'}
    else:            params = {'page_num': pages_num,'view_type': 'list'}
    
    res = requests.get(url_for_mmdict, params=params, headers=headers)
    time.sleep(1)
    
    soup = BeautifulSoup(res.text, 'html.parser')  # создаем обьект bs4.BeautifulSoup
    marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') # список html-ек марок на странице
    
    if not marks_on_page_list: 
#         print ('стоп')
        break
        
    for mark in marks_on_page_list:
        models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
        link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
        mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
        mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
        mark_name = link_for_mark_name[mark_start:mark_end] # получение обозначения марки      
        
        models_list = []
        for model in models_of_mark:
            link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
            model_start = mark_end+1                            # первый символ обозначения модели
            model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
            model_name = link_theme_auto[model_start:model_end] # получение обозначения модели
            models_list.append(model_name)
      
        marks_models_dict[mark_name] = models_list

## Получение данных по одной модели одной марки

In [69]:
mark = list(marks_models_dict.keys())[11]

In [70]:
# mark = 'audi'
models_list = marks_models_dict[mark]
model = models_list[0]

url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'
url

'https://auto.ru/moskva/cars/audi/100/used/'

In [18]:
for pages_num in range(1,4):  # поставить проверку на не пустую страницу
    
    if pages_num==1: params = {}
    else:            params = {'page_num': pages_num}
    
    res = requests.get(url, params=params, headers=headers)
    time.sleep(1)
#     print(res.status_code)
    
    soup = BeautifulSoup(res.text, 'html.parser')  # создаем обьект bs4.BeautifulSoup
    tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') # список html-ек карточек на странице
    
    

200
200
200


In [71]:
# как бы первая страница
params = {} # на страницах будет {'page':page_num}

res = requests.get(url, params=params, headers=headers)
res.encoding = 'utf-8'

soup = BeautifulSoup(res.text, 'html.parser')  # создаем обьект bs4.BeautifulSoup

tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') # список html-ек карточек на странице

In [78]:
# как бы первое обьявление
ticket_url = tickets_on_page_list[0].get('href')
ticket_url

ticket_res = requests.get(ticket_url)
ticket_res.encoding = 'utf-8'

ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')

In [80]:
year = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
year

'1993'

In [81]:
kmAge = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
kmAge = re.sub("\D", "", kmAge)
kmAge

'786115'

In [83]:
bodytype = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
bodytype

'седан'

In [84]:
color = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
color

'синий'

In [100]:
volume = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
volume =  re.sub("[^\d.]", "", volume)
volume

'2.3'

In [92]:
power = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
power = re.sub("\D", "", power)
power

'133'

In [90]:
fuel type = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
fuel type

'Бензин'

In [103]:
transmission = ticket_bs.find('li',class_='CardInfoRow_transmission').find_all('span')[1].text
transmission

'механическая'

In [106]:
drive = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
drive

'передний'

In [108]:
wheel = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text
wheel

'Левый'

In [110]:
state = ticket_bs.find('li',class_='CardInfoRow_state').find_all('span')[1].text
state

'Не требует ремонта'

In [111]:
ownersCount = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
ownersCount

'3 или более'

In [112]:
pts = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
pts

'Оригинал'

In [113]:
customs = ticket_bs.find('li',class_='CardInfoRow_customs').find_all('span')[1].text
customs

'Растаможен'

# Подвал

for loc in tqdm(range(0,len(unuseful_features_LR),1)):

In [8]:
marks = marks_with_counts.split('\n')[::2]
marks

['AC',
 'AMC',
 'Acura',
 'Adler',
 'Alfa Romeo',
 'Alpina',
 'Ariel',
 'Aro',
 'Asia',
 'Aston Martin',
 'Auburn',
 'Audi',
 'Austin',
 'BMW',
 'BYD',
 'Bentley',
 'Borgward',
 'Brabus',
 'Brilliance',
 'Bugatti',
 'Buick',
 'CHERYEXEED',
 'Cadillac',
 'Changan',
 'Chery',
 'Chevrolet',
 'Chrysler',
 'Citroen',
 'DKW',
 'DS',
 'DW Hower',
 'Dacia',
 'Daewoo',
 'Daihatsu',
 'Daimler',
 'Datsun',
 'Delage',
 'Derways',
 'Dodge',
 'DongFeng',
 'Doninvest',
 'Eagle',
 'Excalibur',
 'FAW',
 'Ferrari',
 'Fiat',
 'Fisker',
 'Ford',
 'Foton',
 'GAC',
 'GMC',
 'Geely',
 'Genesis',
 'Great Wall',
 'Hafei',
 'Haima',
 'Haval',
 'Hawtai',
 'Heinkel',
 'Honda',
 'Hummer',
 'Hyundai',
 'Infiniti',
 'Iran Khodro',
 'Isuzu',
 'JAC',
 'Jaguar',
 'Jeep',
 'Kia',
 'LADA (ВАЗ)',
 'LTI',
 'Lamborghini',
 'Lancia',
 'Land Rover',
 'Landwind',
 'Lexus',
 'Lifan',
 'Lincoln',
 'Luxgen',
 'MG',
 'MINI',
 'Mahindra',
 'Maserati',
 'Maybach',
 'Mazda',
 'McLaren',
 'Mercedes-Benz',
 'Mercury',
 'Mitsubishi',
 '

In [21]:
hdrs_office = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'suid=3e1833cc7469d2dc00ccae04449006c4.9491b97d8d423e5ce05ef1c79af8855d; _ym_uid=1599469037495440295; _ga=GA1.2.1600002130.1601280177; _csrf_token=1d7055285f2e0f66d88a7fe1fc75e2eec8ae6cd772464581; autoru_sid=a%3Ag60264c2d2igjpia4p3it7lul5qeu67s.cda7ff5877363e08fe2b0faf7c62fcdd%7C1613122605806.604800.gg8JxiHvE9AyBZihcInqEA.Ld4FRMZg9Xk8q0iFZFqjpOoMCJDw6N7itfobSBJS6WA; autoruuid=g60264c2d2igjpia4p3it7lul5qeu67s.cda7ff5877363e08fe2b0faf7c62fcdd; from=direct; X-Vertis-DC=vla; yuidlt=1; yandexuid=905825121577456505; my=YyYBAS4BAToBAQA%3D; counter_ga_all7=2; gdpr=0; _ym_isad=2; gids=213; _gid=GA1.2.204714820.1613122653; autoru-visits-count=1; from_lifetime=1613124329322; _ym_d=1613124329; cycada=QtZcIspPQofWD6yVq1tXyo1meqCq6CwJcSH8NY94K9c=',
'Host': 'auto.ru',
'Referer': 'https://auto.ru/moskva/cars/used/',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ,Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
},

In [32]:
hdrs_home = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
# 'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
# 'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1612822568824.604800.vXs7yiSNjHUj9vs_NTRT7Q.Ab6eLoCklN5OVQvHKRzq30jYE9puDG83Hht-435SfG4; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; X-Vertis-DC=vla; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; _ym_d=1613218878; from_lifetime=1613218878985; cycada=S1YNtBaS32C5Sgb39LwY/7150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
# 'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
# 'sec-ch-ua-mobile': '?0',
# 'Sec-Fetch-Dest': 'document',
# 'Sec-Fetch-Mode': 'navigate',
# 'Sec-Fetch-Site': 'none',
# 'Sec-Fetch-User': '?1',
# 'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

```python
url = 'https://auto.ru/moskva/cars/audi/100/used/'   
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  
data = soup.find_all('div', class_ ="LayoutSidebar")
df = pd.read_html(str(data))[0]
df```

In [ ]:
def features_parser (url_list):
    counter = 0
    for card_url in url_list:
        response = requests.get(card_url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
    # body type
        body = page.find(itemprop = 'bodyType')
        if body is None:
            data_train.at[counter,'bodyType'] = None
        else:
            data_train.at[counter,'bodyType'] = body['content']
    #brand
        brand = page.find(itemprop = 'brand')
        if brand is None:
            data_train.at[counter, 'brand'] = None
        else:
            data_train.at[counter, 'brand'] = brand['content']
    # color
        color = page.find(itemprop = 'color')
        if color is None:
            data_train.at[counter,'color'] = None
        else:
            data_train.at[counter,'color'] = color['content']
    # fuel type
        fuel = page.find(itemprop = 'fuelType')
        if fuel is None:
            data_train.at[counter,'fuelType'] = None
        else:
            data_train.at[counter,'fuelType'] = fuel['content']
    # model date
        model_date = page.find(itemprop = 'modelDate')
        if model_date is None:
            data_train.at[counter,'modelDate'] = None
        else:
            data_train.at[counter,'modelDate'] = model_date['content']
    # number of doors
        doors = page.find(itemprop = 'numberOfDoors')
        if doors is None:
            data_train.at[counter,'numberOfDoors'] = None
        else:
            data_train.at[counter,'numberOfDoors'] = doors['content']
    # production date
        prod_date = page.find(itemprop = 'productionDate')
        if prod_date is None:
            data_train.at[counter,'productionDate'] = None
        else:
            data_train.at[counter,'productionDate'] = prod_date['content']
    # vehicle transmission
        trans = page.find(itemprop = 'vehicleTransmission')
        if trans is None:
            data_train.at[counter,'vehicleTransmission'] = None
        else:
            data_train.at[counter,'vehicleTransmission'] = trans['content']
    # engine displacement
        engine_ltr = page.find(itemprop = 'engineDisplacement')
        if engine_ltr is None:
            data_train.at[counter,'engineDisplacement'] = None
        else:
            data_train.at[counter,'engineDisplacement'] = engine_ltr['content']
    # engine power
        engine_pwr = page.find(itemprop = 'enginePower')
        if engine_pwr is None:
            data_train.at[counter,'enginePower'] = None
        else:
            data_train.at[counter,'enginePower'] = engine_pwr['content']
    # mileage
        mileage = page.find(class_ = 'CardInfoRow CardInfoRow_kmAge')
        if mileage is None:
            data_train.at[counter,'mileage'] = None
        else:
            mileage = mileage.text.replace('Пробег', '')
            mileage = mileage.replace('\xa0', '')
            mileage = mileage.replace('км', '')
            data_train.at[counter,'mileage'] = mileage
    # drive type
        drive = page.find(class_ = 'CardInfoRow CardInfoRow_drive')
        if drive is None:
            data_train.at[counter,'Привод'] = None
        else:
            drive = drive.text.replace('Привод', '')
            data_train.at[counter,'Привод'] = drive
    # wheel
        wheel = page.find(class_ = 'CardInfoRow CardInfoRow_wheel')
        if wheel is None:
            data_train.at[counter,'Руль'] = None
        else:
            wheel = wheel.text.replace('Руль', '')
            data_train.at[counter,'Руль'] = wheel
    # owners
        owners = page.find(class_ = 'CardInfoRow CardInfoRow_ownersCount')
        if owners is None:
            data_train.at[counter,'Владельцы'] = None
        else:
            owners = owners.text.replace('Владельцы', '')
            owners = owners.replace('\xa0', '')
            data_train.at[counter,'Владельцы'] = owners
    # vehicle certificate
        certificate = page.find(class_ = 'CardInfoRow CardInfoRow_pts')
        if certificate is None:
            data_train.at[counter,'ПТС'] = None
        else:
            certificate = certificate.text.replace('ПТС', '')
            data_train.at[counter,'ПТС'] = certificate
    # price
        price = page.find(class_ = 'OfferPriceCaption__price')
        if price is None:
            data_train.at[counter,'price'] = None
        else:
            price = price.text.replace('\xa0', '')
            price = price.replace('₽', '')
            data_train.at[counter,'price'] = price
    # counter up
        counter += 1

In [20]:
{
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1612822568824.604800.vXs7yiSNjHUj9vs_NTRT7Q.Ab6eLoCklN5OVQvHKRzq30jYE9puDG83Hht-435SfG4; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; X-Vertis-DC=vla; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; _ym_d=1613218878; from_lifetime=1613218878985; cycada=S1YNtBaS32C5Sgb39LwY/7150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'none',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
 'Connection': 'keep-alive',
 'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1612822568824.604800.vXs7yiSNjHUj9vs_NTRT7Q.Ab6eLoCklN5OVQvHKRzq30jYE9puDG83Hht-435SfG4; crookie=uCcId3jGlY

In [2]:
marks_with_counts = '''AC
2
AMC
3
Acura
52
Adler
1
Alfa Romeo
43
Alpina
5
Ariel
1
Aro
1
Asia
2
Aston Martin
14
Auburn
1
Audi
3253
Austin
1
BMW
6114
BYD
11
Bentley
93
Borgward
1
Brabus
1
Brilliance
15
Bugatti
1
Buick
11
CHERYEXEED
12
Cadillac
411
Changan
158
Chery
529
Chevrolet
2144
Chrysler
185
Citroen
834
DKW
2
DS
1
DW Hower
5
Dacia
3
Daewoo
538
Daihatsu
26
Daimler
3
Datsun
121
Delage
1
Derways
2
Dodge
237
DongFeng
12
Doninvest
3
Eagle
1
Excalibur
1
FAW
47
Ferrari
29
Fiat
163
Fisker
1
Ford
2762
Foton
1
GAC
10
GMC
18
Geely
296
Genesis
35
Great Wall
241
Hafei
1
Haima
11
Haval
514
Hawtai
1
Heinkel
1
Honda
739
Hummer
47
Hyundai
4698
Infiniti
658
Iran Khodro
15
Isuzu
23
JAC
5
Jaguar
243
Jeep
457
Kia
4842
LADA (ВАЗ)
6537
LTI
1
Lamborghini
10
Lancia
5
Land Rover
1214
Landwind
1
Lexus
1360
Lifan
171
Lincoln
45
Luxgen
7
MG
6
MINI
554
Mahindra
1
Maserati
47
Maybach
5
Mazda
1892
McLaren
2
Mercedes-Benz
10162
Mercury
9
Mitsubishi
2660
Mitsuoka
1
Nissan
4375
Oldsmobile
6
Opel
1968
Packard
2
Peugeot
1017
Plymouth
12
Pontiac
25
Porsche
613
Proton
3
RAM
17
Ravon
68
Renault
3878
Rolls-Royce
51
Rover
40
SEAT
56
Saab
86
Saturn
6
Scion
5
Shanghai Maple
1
Skoda
2897
Smart
74
SsangYong
413
Subaru
576
Suzuki
1055
Tatra
1
Tesla
60
Tianma
1
Toyota
3087
Triumph
1
Volkswagen
5541
Volvo
1153
Vortex
44
Willys
2
Xin Kai
1
ZX
5
Zotye
5
ГАЗ
683
Гоночный автомобиль
3
ЗАЗ
178
ЗИЛ
3
ЗиС
1
ИЖ
77
ЛуАЗ
33
Москвич
145
СМЗ
4
ТагАЗ
46
УАЗ
781'''

In [3]:
'https://auto.ru/catalog/cars/used/?view_type=list'
'https://auto.ru/catalog/cars/used/?page_num=2&view_type=list'

'https://auto.ru/catalog/cars/used/?page_num=2&view_type=list'

In [ ]:
for mark in marks_on_page_list:
    models_of_mark = mark.find_all('a', class_='link_theme_auto') # html списка моделей марки
    
    link_for_mark_name = models_of_mark[0].get('href')  # ссылка на первую модель, содержит обозначене марки на auto.ru
    mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
    mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
    mark_name = link_for_mark_name[mark_start:mark_end] # получение обозначения марки
    
    print(mark_name, len(models_of_mark))
    
    for model in models_of_mark:
        link_theme_auto = model.get('href')
        model_start = mark_end+1
        model_end = link_theme_auto.find('/', model_start)
        model_name = link_theme_auto[model_start:model_end]
        print('\t',model_name)

In [ ]:
params_first_page = {
    'view_type': 'list'
}
res = re.get(url, params=params_first_page, headers=headers)

pages_num = 2

params_next_pages = {
    'page_num': pages_num,
    'view_type': 'list'
}

res_2 = re.get(url, params=params_next_pages, headers=headers)

res.status_code,res_2.status_code

soup = BeautifulSoup(res.text, 'html.parser')
soup_2 = BeautifulSoup(res_2.text, 'html.parser')

In [ ]:
marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc')

models_of_mark = marks_on_page_list[1].find_all('a', class_='link_theme_auto')

link_theme_auto = models_of_mark[0].get('href')
mark_start = link_theme_auto.find('cars/') + 5
mark_end = link_theme_auto.find('/', mark_start)
model_start = mark_end+1
model_end = link_theme_auto.find('/', model_start)

link_theme_auto[mark_start:mark_end], link_theme_auto[model_start:model_end]